In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
%matplotlib inline

# Assuming your data files are saved in "(your current directory)/io/" folder. 

In [2]:
train = pd.read_csv("train.csv")
train.head()

,Donation ID,Donor ID,Donation Amount,Donation Received Date,# of Donations Made,min,max,Total Sum Donated,Donor State,Donor Is Teacher,...,School State,# Donations Received,Total Donation Amt Received,Teacher Project Posted Sequence,Project Title,Project Subject Category Tree,Project Subject Subcategory Tree,Project Resource Category,Project Current Status,Resource Vendor Name
0,579821d4de7922dd32ae6aa9f1e67fd9,00070fd37d3e8b5b7c08a4a0ba22aba1,50.00,2016-01-18 18:40:29,29,10.00,150.0,1560.00,Oklahoma,No,...,Oklahoma,8,375.00,5,Science Lab Supplies,Math & Science,"Applied Sciences, Health & Life Science",Supplies,Expired,"Carolina Biological Supply Company,Carolina Bi..."
1,f2bbdba4619801071b2e97a483ff2c71,00070fd37d3e8b5b7c08a4a0ba22aba1,50.00,2017-09-17 12:46:24,29,10.00,150.0,1560.00,Oklahoma,No,...,Oklahoma,6,370.13,2,Your Epidermis is Showing...Your Art!,Music & The Arts,Visual Arts,Supplies,Fully Funded,"Amazon Business,Amazon Business"
2,886fdf2feda0118c8f2adcac7b15b0e4,0015dc157335e5ad6f569e5264bbf33e,50.00,2016-01-10 10:11:22,22,10.00,2700.0,4505.86,Maryland,Yes,...,Maryland,5,718.28,20,Shine Some Light on the Subject!,"Special Needs, Music & The Arts","Special Needs, Visual Arts",Supplies,Fully Funded,"Amazon Business,Amazon Business,Amazon Busines..."
3,02216d6d22189b292fd6285285b95c12,0015dc157335e5ad6f569e5264bbf33e,41.00,2013-04-24 16:23:11,22,10.00,2700.0,4505.86,Maryland,Yes,...,Maryland,2,143.16,11,Repetition With Excitement!,"Applied Learning, Special Needs","Early Development, Special Needs",Books,Fully Funded,"AKJ Education,AKJ Education,AKJ Education,AKJ ..."
4,7798f746c2956bfd29f394b5b127e6a4,00199e3565635f8a5ebefd3b5985a7f3,238.65,2018-01-25 08:57:10,215,11.24,1116.4,46502.98,Massachusetts,No,...,Massachusetts,1,238.65,3,Teaching Kids Sports,Health & Sports,"Health & Wellness, Team Sports",Sports & Exercise Equipment,Fully Funded,"School Specialty,Staples Advantage,School Spec..."


## Cosine Similarity 

In [3]:
# list1v = train.columns.values.tolist()
# print(list1v)

cosimilar_df = train[['Donor ID', 'Project ID','Donation ID','Donation Amount','Project Category- Applied Learning', 'Project Category- Health & Sports', 'Project Category- History & Civics', 'Project Category- Literacy & Language', 'Project Category- Math & Science', 'Project Category- Music & The Arts','Project Category- Special Needs', 'Project Category- Warmth, Care & Hunger']]
cosimilar_df.head()

import math

def smooth_donor_preference(x):
    return math.log(1+x, 2)
    
cosine_df_donatestrength = cosimilar_df \
                    .groupby(['Donor ID', 'Project ID','Donation ID'])['Donation Amount'].sum() \
                    .apply(smooth_donor_preference).reset_index()

cosine_df_donatestrength.head()
cosine_df_donatestrength.columns = ['Donor ID', 'Project ID','Donation ID','Donation Strength']
cosine_df1 = pd.merge(cosimilar_df, right=cosine_df_donatestrength, on=["Donor ID","Project ID",'Donation ID'], how="inner")
variables_drop = ['Donation Amount','Project ID','Donation ID']
cosine_df2 = cosine_df1[cosine_df1.columns.drop(variables_drop)]
variable_df_multiple = cosine_df2[['Project Category- Applied Learning', 'Project Category- Health & Sports', 'Project Category- History & Civics', 'Project Category- Literacy & Language', 'Project Category- Math & Science', 'Project Category- Music & The Arts','Project Category- Special Needs', 'Project Category- Warmth, Care & Hunger']].multiply(cosine_df2['Donation Strength'], axis="index")
cosine_df3 = pd.concat([cosine_df2['Donor ID'],variable_df_multiple],axis=1)
cosine_df4 = cosine_df3.groupby(['Donor ID']).sum().reset_index()
cosine_df5 = cosine_df4[cosine_df4.columns.drop(['Donor ID'])]
cosine_df5.head()

,Project Category- Applied Learning,Project Category- Health & Sports,Project Category- History & Civics,Project Category- Literacy & Language,Project Category- Math & Science,Project Category- Music & The Arts,Project Category- Special Needs,"Project Category- Warmth, Care & Hunger"
0,0.000000,0.000000,0.0,0.000000,5.672425,5.672425,0.000000,0.0
1,5.392317,0.000000,0.0,0.000000,0.000000,5.672425,11.064743,0.0
2,0.000000,39.638098,0.0,0.000000,0.000000,0.000000,0.000000,0.0
3,0.000000,0.000000,0.0,5.672425,13.316423,6.658211,6.658211,0.0
4,0.000000,0.000000,0.0,5.392317,5.392317,0.000000,0.000000,0.0


In [4]:
# cosimilar_pivot_subj = variable_df.pivot_table(index='Donor ID',columns=['Project Category- Applied Learning', 'Project Category- Health & Sports', 'Project Category- History & Civics', 'Project Category- Literacy & Language', 'Project Category- Math & Science', 'Project Category- Music & The Arts','Project Category- Special Needs', 'Project Category- Warmth, Care & Hunger'],values='Donation Strength',aggfunc='sum')
# cosimilar_pivot_resour = cosimilar_df.pivot_table(index='Donor ID',columns='Project Resource Category',values='Donation Strength',aggfunc='mean')
# cosimilar_pivot_metro = cosimilar_df.pivot_table(index='Donor ID',columns='School Metro Type',values='Donation Strength',aggfunc='mean')
# cosimilar_merge1 = pd.merge(left=cosimilar_pivot_subj, right=cosimilar_pivot_resour, on='Donor ID', how="inner")
# cosimilar_merge2 = pd.merge(left=cosimilar_merge1, right=cosimilar_pivot_metro, on='Donor ID', how="inner")
# cosimilar_pivot_subj1 = cosimilar_pivot_subj.fillna(0)

# cosimilar_pivot_subj1.head()

In [5]:
from scipy import sparse
cosine_df_sparse = sparse.csr_matrix(cosine_df5)
# print(cosine_df_sparse)

In [6]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_df_similarities = cosine_similarity(cosine_df_sparse)
cosine_df_similarities_sparse = sparse.csr_matrix(cosine_df_similarities)
print(cosine_df_similarities_sparse)

  (0, 0)	0.9999999999999998
  (0, 1)	0.29595143959370823
  (0, 3)	0.8179636209368781
  (0, 4)	0.5
  (0, 5)	0.8164965809277259
  (0, 6)	0.36433168221519663
  (0, 7)	0.43115228710352826
  (0, 8)	0.6097558353757594
  (0, 9)	0.3788422613828998
  (0, 10)	0.7071067811865475
  (0, 11)	0.8597258107417951
  (0, 12)	0.12684251632611132
  (0, 13)	0.5418100575933782
  (0, 15)	0.793960914873322
  (0, 16)	0.3214501838443423
  (0, 19)	0.2981873551110751
  (0, 20)	0.31622776601683783
  (0, 21)	0.39871206349823296
  (0, 23)	0.43832322011685226
  (0, 24)	0.17509309524979164
  (0, 26)	0.5428179529791463
  (0, 28)	0.8017837257372731
  (0, 29)	0.6743802172901854
  (0, 30)	0.408248290463863
  (0, 31)	0.5217016862479367
  :	:
  (8784, 8746)	0.5
  (8784, 8747)	0.5486826666714103
  (8784, 8748)	0.605692619619863
  (8784, 8753)	0.40910914387809466
  (8784, 8754)	0.22614876586156765
  (8784, 8755)	0.9214079835812045
  (8784, 8757)	0.8403974994111616
  (8784, 8759)	0.6702050965242221
  (8784, 8760)	0.972816944257

In [7]:
donorsim_df1 = pd.DataFrame(cosine_df_similarities_sparse.todense())
donorsim_df1['Donor'] = donorsim_df1.index
donorsim_df1.head()

,0,1,2,3,4,5,6,7,8,9,...,8776,8777,8778,8779,8780,8781,8782,8783,8784,Donor
0,1.000000,0.295951,0.0,0.817964,0.500000,0.816497,0.364332,0.431152,0.609756,0.378842,...,0.211343,0.394591,0.295369,0.437516,0.000000,0.499471,0.500000,0.000000,0.707107,0
1,0.295951,1.000000,0.0,0.476186,0.000000,0.241643,0.488140,0.000000,0.360916,0.191842,...,0.125095,0.455585,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1
2,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.686562,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.723256,0.000000,0.000000,0.000000,0.000000,2
3,0.817964,0.476186,0.0,1.000000,0.777596,0.857526,0.367932,0.550307,0.498844,0.770199,...,0.428734,0.847275,0.620605,0.735233,0.226857,0.544732,0.777596,0.290721,0.771183,3
4,0.500000,0.000000,0.0,0.777596,1.000000,0.816497,0.364332,0.603535,0.358047,0.952314,...,0.674784,0.828868,0.937831,0.993016,0.488314,0.499471,1.000000,0.625781,0.707107,4


In [8]:
donor_namelist = cosine_df4[['Donor ID']]
donor_namelist['Donor'] = donor_namelist.index + 1
donor_namelist.head()

c:\users\me\desktop\pythonapp\testprojs\venv\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Donor ID,Donor
0,00070fd37d3e8b5b7c08a4a0ba22aba1,1
1,0015dc157335e5ad6f569e5264bbf33e,2
2,00199e3565635f8a5ebefd3b5985a7f3,3
3,00200ccc822140acfc2f15dc809d15e5,4
4,0026a8fc3a4faea82ccab26c6f55b709,5


In [9]:
donorsim_df2 = pd.melt(donorsim_df1,id_vars='Donor')
donorsim_df2.Donor = donorsim_df2.Donor + 1
donorsim_df2.variable = donorsim_df2.variable + 1
donorsim_df3 = donorsim_df2[donorsim_df2.value != 0]
donorsim_df4 = donorsim_df3[donorsim_df3['variable'] != donorsim_df3['Donor']]
donorsim_df5 = pd.merge(left=donorsim_df4, right=donor_namelist, on="Donor", how="inner")
donorsim_df6 = donorsim_df5[donorsim_df5.columns.drop(['Donor'])]
donorsim_df6.columns = ['Donor', 'Similarity Score', 'Original Donor ID']
donorsim_df7 = pd.concat([donorsim_df6,donor_namelist], axis=1, sort=False)
cols = [0,4]
donorsim_df7.drop(donorsim_df7.columns[cols],axis=1,inplace=True)
donorsim_df7.columns = ['Similarity Score', 'Original Donor ID','Other Donor ID']
donorsim_df7.head()

,Similarity Score,Original Donor ID,Other Donor ID
0,0.295951,0015dc157335e5ad6f569e5264bbf33e,00070fd37d3e8b5b7c08a4a0ba22aba1
1,0.476186,0015dc157335e5ad6f569e5264bbf33e,0015dc157335e5ad6f569e5264bbf33e
2,0.241643,0015dc157335e5ad6f569e5264bbf33e,00199e3565635f8a5ebefd3b5985a7f3
3,0.488140,0015dc157335e5ad6f569e5264bbf33e,00200ccc822140acfc2f15dc809d15e5
4,0.360916,0015dc157335e5ad6f569e5264bbf33e,0026a8fc3a4faea82ccab26c6f55b709


In [120]:
donorsim_df7[donorsim_df7['Other Donor ID'] == '009d5fc7b87883ffad248db5150bf1fc']

,Similarity Score,Original Donor ID,Other Donor ID
15,0.379582,0015dc157335e5ad6f569e5264bbf33e,009d5fc7b87883ffad248db5150bf1fc


In [10]:
donorproj_df1 = cosine_df1.groupby(['Donor ID','Project ID'])['Donation Amount'].agg(['count'])
donorproj_df2 = donorproj_df1[donorproj_df1.columns.drop(['count'])]
donorproj_df2.tail()

Empty DataFrame
Columns: []
Index: [(fff99b7b56af3835b317bc1233bb9595, 1e9b3f44be493f155067e8261fcd34ef), (fff99b7b56af3835b317bc1233bb9595, 5a4a394d00fb96df0bc43006d3a2151c), (fff99b7b56af3835b317bc1233bb9595, 6820b430838c5dc51690a4d538d25749), (fffd69c0f1dc11f7fc10d595891723a2, 4eb5d72bfd1e1577536267269e7aee19), (fffd69c0f1dc11f7fc10d595891723a2, 81426170e4ee9b1d27b8316a95b93703)]

In [11]:
# donorsim_df7.to_csv('donorsim.csv', encoding='utf-8', index=False)
# donorproj_df1.to_csv('donorproj.csv', encoding='utf-8', index=False)

In [12]:
test = pd.read_csv("test.csv")
testset = test.groupby(['Donor ID','Project ID'])['Donation Amount'].agg(['count'])
testset2 = testset[testset.columns.drop(['count'])]
testset2.head()

Empty DataFrame
Columns: []
Index: [(006c2d57cca011d6040fed3a2ceb8179, dd19fb9ec77a69b6e9ffd956a3a88b5b), (009d5fc7b87883ffad248db5150bf1fc, 0c7d99a1079bb22a500e71a7647a54b6), (009d5fc7b87883ffad248db5150bf1fc, 13ffe6f614ee069f1a6faa303bd5a94e), (009d5fc7b87883ffad248db5150bf1fc, 958909abb8551c487f34570b2ded43d6), (009d5fc7b87883ffad248db5150bf1fc, c5ce8362cc1ea4ec1f99ba880252de63)]

In [107]:
#test set
testset3 = testset2.reset_index()
#project set
testset4 = donorproj_df2.reset_index()
#score set
scoreset = donorsim_df7.set_index('Other Donor ID').copy()

list_of_donor_ids = testset3['Donor ID'].values.tolist()

len(list_of_donor_ids)

list_of_donor_ids


check_df = scoreset.loc[list(set(list_of_donor_ids))].copy()

check_df.describe()

check_df.head()

,Similarity Score,Original Donor ID
Other Donor ID,,
be833db4f16df25696356b1194a14308,0.426258,00200ccc822140acfc2f15dc809d15e5
115471ea6d09d530d15afbac5962b4f3,0.561547,0015dc157335e5ad6f569e5264bbf33e
2bd0e89b88aa24b47ce92989ad7d2735,0.185699,0015dc157335e5ad6f569e5264bbf33e
35372ef3b7e130b25a7d04e3bc5f83c8,0.309103,0015dc157335e5ad6f569e5264bbf33e
46d09ff429014b52316d88db2b37cd41,0.653999,0015dc157335e5ad6f569e5264bbf33e


In [ ]:
check_df.loc[]

In [108]:
joined = check_df.join(testset4.set_index('Donor ID'))

joined.describe()

joined.head()

,Similarity Score,Original Donor ID,Project ID
006c2d57cca011d6040fed3a2ceb8179,0.462709,0015dc157335e5ad6f569e5264bbf33e,dd19fb9ec77a69b6e9ffd956a3a88b5b
009d5fc7b87883ffad248db5150bf1fc,0.379582,0015dc157335e5ad6f569e5264bbf33e,022a1517e63f4e66845358f15ff7793e
009d5fc7b87883ffad248db5150bf1fc,0.379582,0015dc157335e5ad6f569e5264bbf33e,02f2f8fc72b4714f5c6b98c04edb5913
009d5fc7b87883ffad248db5150bf1fc,0.379582,0015dc157335e5ad6f569e5264bbf33e,0378b6a177fd71d4416b764538b9daa8
009d5fc7b87883ffad248db5150bf1fc,0.379582,0015dc157335e5ad6f569e5264bbf33e,07c5fc3f0b880304cefb39b3c43a34ed


In [111]:
testset4.set_index('Donor ID').loc['009d5fc7b87883ffad248db5150bf1fc']

,Project ID
Donor ID,
009d5fc7b87883ffad248db5150bf1fc,022a1517e63f4e66845358f15ff7793e
009d5fc7b87883ffad248db5150bf1fc,02f2f8fc72b4714f5c6b98c04edb5913
009d5fc7b87883ffad248db5150bf1fc,0378b6a177fd71d4416b764538b9daa8
009d5fc7b87883ffad248db5150bf1fc,07c5fc3f0b880304cefb39b3c43a34ed
009d5fc7b87883ffad248db5150bf1fc,08012e5f6db6ef1b7b4ed8f1887ec992
009d5fc7b87883ffad248db5150bf1fc,0814ec4389bfe0f303381eb138b872d6
009d5fc7b87883ffad248db5150bf1fc,0b3153c1a5b068d87c0ba18422e9f15f
009d5fc7b87883ffad248db5150bf1fc,0c7d99a1079bb22a500e71a7647a54b6
009d5fc7b87883ffad248db5150bf1fc,0db2e61d82f151ce2de511ea8c0d349c


In [112]:
joined[joined.isnull().any(axis=1)]

,Similarity Score,Original Donor ID,Project ID


In [113]:
joined.loc['009d5fc7b87883ffad248db5150bf1fc']

,Similarity Score,Original Donor ID,Project ID
009d5fc7b87883ffad248db5150bf1fc,0.379582,0015dc157335e5ad6f569e5264bbf33e,022a1517e63f4e66845358f15ff7793e
009d5fc7b87883ffad248db5150bf1fc,0.379582,0015dc157335e5ad6f569e5264bbf33e,02f2f8fc72b4714f5c6b98c04edb5913
009d5fc7b87883ffad248db5150bf1fc,0.379582,0015dc157335e5ad6f569e5264bbf33e,0378b6a177fd71d4416b764538b9daa8
009d5fc7b87883ffad248db5150bf1fc,0.379582,0015dc157335e5ad6f569e5264bbf33e,07c5fc3f0b880304cefb39b3c43a34ed
009d5fc7b87883ffad248db5150bf1fc,0.379582,0015dc157335e5ad6f569e5264bbf33e,08012e5f6db6ef1b7b4ed8f1887ec992
009d5fc7b87883ffad248db5150bf1fc,0.379582,0015dc157335e5ad6f569e5264bbf33e,0814ec4389bfe0f303381eb138b872d6
009d5fc7b87883ffad248db5150bf1fc,0.379582,0015dc157335e5ad6f569e5264bbf33e,0b3153c1a5b068d87c0ba18422e9f15f
009d5fc7b87883ffad248db5150bf1fc,0.379582,0015dc157335e5ad6f569e5264bbf33e,0c7d99a1079bb22a500e71a7647a54b6
009d5fc7b87883ffad248db5150bf1fc,0.379582,0015dc157335e5ad6f569e5264bbf33e,0db2e61d82f151ce2de511ea8c0d349c
009d5fc7b87883ffad248db5150bf1fc,0.379582,0015dc157335e5ad6f569e5264bbf33e,0dd554bf40e1fbfe3e366755094becd4


In [114]:
scoreset.loc['009d5fc7b87883ffad248db5150bf1fc']

Similarity Score                             0.379582
Original Donor ID    0015dc157335e5ad6f569e5264bbf33e
Name: 009d5fc7b87883ffad248db5150bf1fc, dtype: object

In [115]:
joined

,Similarity Score,Original Donor ID,Project ID
006c2d57cca011d6040fed3a2ceb8179,0.462709,0015dc157335e5ad6f569e5264bbf33e,dd19fb9ec77a69b6e9ffd956a3a88b5b
009d5fc7b87883ffad248db5150bf1fc,0.379582,0015dc157335e5ad6f569e5264bbf33e,022a1517e63f4e66845358f15ff7793e
009d5fc7b87883ffad248db5150bf1fc,0.379582,0015dc157335e5ad6f569e5264bbf33e,02f2f8fc72b4714f5c6b98c04edb5913
009d5fc7b87883ffad248db5150bf1fc,0.379582,0015dc157335e5ad6f569e5264bbf33e,0378b6a177fd71d4416b764538b9daa8
009d5fc7b87883ffad248db5150bf1fc,0.379582,0015dc157335e5ad6f569e5264bbf33e,07c5fc3f0b880304cefb39b3c43a34ed
009d5fc7b87883ffad248db5150bf1fc,0.379582,0015dc157335e5ad6f569e5264bbf33e,08012e5f6db6ef1b7b4ed8f1887ec992
009d5fc7b87883ffad248db5150bf1fc,0.379582,0015dc157335e5ad6f569e5264bbf33e,0814ec4389bfe0f303381eb138b872d6
009d5fc7b87883ffad248db5150bf1fc,0.379582,0015dc157335e5ad6f569e5264bbf33e,0b3153c1a5b068d87c0ba18422e9f15f
009d5fc7b87883ffad248db5150bf1fc,0.379582,0015dc157335e5ad6f569e5264bbf33e,0c7d99a1079bb22a500e71a7647a54b6
009d5fc7b87883ffad248db5150bf1fc,0.379582,0015dc157335e5ad6f569e5264bbf33e,0db2e61d82f151ce2de511ea8c0d349c


In [116]:
mean_score = joined.groupby('Project ID').mean().copy()

threshold = 0.5

mean_score['indicator'] = np.where(mean_score['Similarity Score'] > threshold, 1, 0)

mean_score.describe()

,Similarity Score,indicator
count,6609.000000,6609.000000
mean,0.437860,0.346951
std,0.208074,0.476036
min,0.028002,0.000000
25%,0.273762,0.000000
50%,0.387773,0.000000
75%,0.563386,1.000000
max,0.987247,1.000000


In [117]:
project_ids_above_thres = mean_score[mean_score['indicator'] ==  1].copy()
print(project_ids_above_thres.head())
joined_project_1 = project_ids_above_thres.join(testset3.set_index('Project ID'))

joined_project_1 = joined_project_1.dropna()

                                  Similarity Score  indicator
Project ID                                                   
0001d58fba851d06ecb8a14853cfe257          0.862209          1
00332491c9662b6327ae47d35a36be49          0.689767          1
00338491afb8901ec167a56d2f2d66ac          0.858626          1
0072118edd7e0c5e84d23be2424bebd9          0.703181          1
00880f1895f61d267078e745bd9657ab          0.536364          1


In [118]:
project_ids_below_thres = mean_score[mean_score['indicator'] ==  0].copy()

joined_project_2 = project_ids_below_thres.join(testset3.set_index('Project ID'))

joined_project_2 = joined_project_2.dropna()
joined_project_2.head()

,Similarity Score,indicator,Donor ID
Project ID,,,
002ad9789f045ac1a55148fb60143b2b,0.322343,0,8e04337ac2c7de84ebf95ce5ebc6b832
0055f22141ae2bec81abcd922c8247fe,0.328503,0,c89faede8cc42c9f695348154c8796f6
009df62864367965fd4e74786270f28d,0.401101,0,65f546d879e1d67ea5442924110bc400
00e29987b8f724f49bed5b0c090fe9bc,0.093680,0,2adabd4396dee7e54b553cddb30a6f87
0168d58daeae4f0d01e2def61e468d4b,0.452067,0,22f51fc8e5750e45e7a3da489eacfe40


In [123]:
joined_project_2[joined_project_2['Donor ID']=='009d5fc7b87883ffad248db5150bf1fc']

,Similarity Score,indicator,Donor ID
Project ID,,,
0c7d99a1079bb22a500e71a7647a54b6,0.414731,0,009d5fc7b87883ffad248db5150bf1fc
13ffe6f614ee069f1a6faa303bd5a94e,0.379582,0,009d5fc7b87883ffad248db5150bf1fc
958909abb8551c487f34570b2ded43d6,0.354042,0,009d5fc7b87883ffad248db5150bf1fc
c5ce8362cc1ea4ec1f99ba880252de63,0.379582,0,009d5fc7b87883ffad248db5150bf1fc


    for 'Donor ID' in testset:
    if 'Donor ID' = 'Original Donor ID' in donorsim_df7, select 'Other Donor ID' 
    
    where 'Other Donor ID' in donorsim_df7 = 'Donor ID' in donorproj_df2, select 'Project ID' and attach 'Similarity Score' 
    
    Compute mean simlarity score for each project
    
    where mean similarity score > threshold, indicate 1, otherwise 0
    
    compare with test set to determine number of 'Donor ID' donations to projects indicated as 1 and 0